In [1]:
import pandas as pd

from evaluation_helper import get_time_and_rss, get_max_gpu_usage

In [2]:
guppy_log_dir = '../../logs/guppy'
minimap_log_dir = '../../logs/minimap'
res_dir = '../../data/eval'
minimap_res_dir = '../../data/eval/minimap'

In [3]:
measures = pd.DataFrame()
for ds in ['real', 'sim']:
    for mx in [4, 6, 8]:
        # add guppy results
        guppy_logfile_time = f'{guppy_log_dir}/step6c_basecall_{ds}_{mx}.txt'
        guppy_logfile_nvidia = f'{guppy_log_dir}/step6c_basecall_{ds}_{mx}_gpu.txt'
        g_user_time, g_system_time, g_elapsed_time, g_max_rss = get_time_and_rss(guppy_logfile_time)
        max_gpu_usage = get_max_gpu_usage(guppy_logfile_nvidia, 'guppy_basecaller')
        measures = pd.concat([measures, pd.DataFrame([{'Approach': 'Guppy',
                                                       'Dataset': ds,
                                                       'Maximum Sequence Length': mx * 1000,
                                                       'User Time': g_user_time,
                                                       'System Time': g_system_time,
                                                       'Elapsed Time': g_elapsed_time,
                                                       'Max RSS (GB)': g_max_rss,
                                                       'Max GPU Memory Usage (GiB)': max_gpu_usage}])], ignore_index=True)

        # add minimap results
        minimap_logfile_time = f'{minimap_log_dir}/step6e_map_{ds}_{mx}.txt'
        m_user_time, m_system_time, m_elapsed_time, m_max_rss = get_time_and_rss(minimap_logfile_time)
        measures = pd.concat([measures, pd.DataFrame([{'ID': f'max{mx}_{ds}',
                                                       'Approach': 'Minimap',
                                                       'Dataset': ds,
                                                       'Maximum Sequence Length': mx * 1000,
                                                       'User Time': m_user_time,
                                                       'System Time': m_system_time,
                                                       'Elapsed Time': m_elapsed_time,
                                                       'Max RSS (GB)': m_max_rss,
                                                       'Max GPU Memory Usage (GiB)': 0.0}])], ignore_index=True)

In [4]:
measures['User Time'] = pd.to_timedelta(measures['User Time'])
measures['System Time'] = pd.to_timedelta(measures['System Time'])
measures['Elapsed Time'] = '00:' + measures['Elapsed Time']
measures['Elapsed Time'] = pd.to_timedelta(measures['Elapsed Time'])

In [ ]:
summed_measures = measures.groupby(['Dataset', 'Maximum Sequence Length'])['User Time', 'System Time', 'Elapsed Time', 'Max RSS (GB)', 'Max GPU Memory Usage (GiB)'].apply(lambda x : x.sum())
summed_measures = summed_measures.reset_index()
summed_measures['Approach'] = 'Guppy + Minimap'
measures = pd.concat([measures, summed_measures], ignore_index=True)
measures

In [6]:
measures.to_csv(f'{res_dir}/MINIMAP_times_and_measures.csv', index=False)

In [ ]:
def extract_ref_names(ref_path, txt_path):
    with open(ref_path, 'r') as f_in, open(txt_path, 'w') as f_out:
        for line in f_in.readlines():
            if ">" in line:
                f_out.write(f'{line.split(" ")[0][1:]}\n')

extract_ref_names(f'{minimap_res_dir}/real_pos_references.fasta', f'{minimap_res_dir}/real_pos_reference_names.txt')
extract_ref_names(f'{minimap_res_dir}/real_neg_references.fasta', f'{minimap_res_dir}/real_neg_reference_names.txt')
extract_ref_names(f'{minimap_res_dir}/sim_pos_references.fasta', f'{minimap_res_dir}/sim_pos_reference_names.txt')
extract_ref_names(f'{minimap_res_dir}/sim_neg_references.fasta', f'{minimap_res_dir}/sim_neg_reference_names.txt')

In [ ]:
def assign_class_to_ref(ref_name, pos_refs, neg_refs):
    if ref_name in pos_refs:
        return 'pos'
    elif ref_name in neg_refs:
        return 'neg'
    else:
        raise ValueError(f'Reference name "{ref_name}" not known!')

In [ ]:
metrics = pd.DataFrame(columns=['ID', 'Maximum Sequence Length', 'Dataset', 'TP', 'TN', 'FP', 'FN', 'UP', 'UC'])

for ds in ['real', 'sim']:
    neg_refs = open(f'{minimap_res_dir}/{ds}_neg_reference_names.txt').read().splitlines()
    pos_refs = open(f'{minimap_res_dir}/{ds}_pos_reference_names.txt').read().splitlines()

    for mx in [4, 6, 8]:
        merged = pd.DataFrame()
        unclassified_plasmids, unclassified_chromosomes = 0, 0
        ids = pd.read_csv(f'{res_dir}/max{mx}_gt_test_{ds}_labels.csv')

        for cls in ['pos', 'neg']:
            alignment = pd.read_csv(f'{minimap_res_dir}/{ds}_max{mx}/{cls}_reads_and_refs.csv', sep='\t', names=['Read', 'Reference', 'Quality'])
            alignment['Read Class'] = cls  # ground truth label
            alignment['Reference Class'] = alignment['Reference'].apply(lambda ref: assign_class_to_ref(ref, pos_refs, neg_refs))  # predicted label
            merged = pd.concat([merged, alignment], ignore_index=True)

            all_reads = len(ids[ids['GT Label'] == f'{"plasmid" if cls == "pos" else "chr"}'])
            matched_reads = len(alignment)
            if cls == 'pos':
                unclassified_plasmids = all_reads - matched_reads
            else:
                unclassified_chromosomes = all_reads - matched_reads

        metrics = pd.concat([metrics, pd.DataFrame([{
            'ID': f'max{mx}_{ds}',
            'Maximum Sequence Length': int(mx) * 1000,
            'Dataset': ds,
            'TP': len(merged[(merged['Reference Class'] == 'pos') & (merged['Read Class'] == 'pos')]),
            'TN': len(merged[(merged['Reference Class'] == 'neg') & (merged['Read Class'] == 'neg')]),
            'FP': len(merged[(merged['Reference Class'] == 'pos') & (merged['Read Class'] == 'neg')]),
            'FN': len(merged[(merged['Reference Class'] == 'neg') & (merged['Read Class'] == 'pos')]),
            'UP': unclassified_plasmids,
            'UC': unclassified_chromosomes,
        }])], ignore_index=True)

metrics['TPR (Sensitivity)'] = metrics['TP'] / (metrics['TP'] + metrics['FN'] + metrics['UP'])
metrics['TNR (Specificity)'] = metrics['TN'] / (metrics['TN'] + metrics['FP'] + metrics['UC'])
metrics['FPR'] = 1 - metrics['TNR (Specificity)']
metrics['FNR'] = 1 - metrics['TPR (Sensitivity)']
metrics['Balanced Accuracy'] = (metrics['TPR (Sensitivity)'] + metrics['TNR (Specificity)']) / 2
metrics['Accuracy'] = (metrics['TP'] + metrics['TN']) / (metrics['TP'] + metrics['TN'] + metrics['FP'] + metrics['FN'] + metrics['UP'] + metrics['UC'])
metrics

In [10]:
metrics.to_csv(f'{res_dir}/MINIMAP_metrics.csv', index=False)